# Task 8 - Optimal Asymetric Encryption Padding (OAEP)
Felix Kleinsteuber $\cdot$ Matrikelnr.: 185709

In [1]:
from hashlib import sha1
import numpy as np

## 1. Mask Generating Function (MGF1)

In [43]:
def mgf1(seed: bytearray, l: int, hash_func) -> bytearray:
    """Mask Generating Function MGF1. Generiert eine l-Byte-Maske aus seed.

    Args:
        seed (bytearray): Startwert beliebiger Länge.
        l (int): Ziellänge der Maske.
        hash (function): Zu verwendende Hashfunktion (bytearray -> bytearray).

    Returns:
        bytearray: Maske T.
    """
    T = bytearray()
    counter = 0
    while len(T) < l:
        counter += 1
        # Baue seed || C
        to_hash = bytearray(seed)
        for off in range(24, -1, -8):
            to_hash.append((counter >> off) & 0xff)
        # T = T || hash(seed || C)
        T += hash_func(to_hash)
    return T[:l]

def hash_func(input: bytes) -> bytes:
    return sha1(input).digest()

# Test
seed = b"hallo!"
l = 320
mask = mgf1(seed, l, hash_func)
assert len(mask) == l
mask.hex()

'4786f2d952d8a72913c1c6f45b7db57eb881262c32056f41dbd8fa889a16be129d6d2bf1876c070a1b525960bc598e1aa33f4456cb8089501e54cd7946f9c164c213baddc3fd2489ba12e63b06b145c7847d3d25e2e6fe50ceba71502d49232db98e958c9c3b7564779f3850fd90233b0809334af8b189bd4064c38f57b88027daa616b8d21c822aa6596b6227c0460916b025d982b0abedd95cf3d4ee2dfce953aba1d366c41937cdd07dc44d5f950bbbfd61421698ccc6bd40fec0b62fa6cb653bc2812ced15a93d813bd85fdf7205ac319617b3b5abeb122ce1f320e30201a0e4006f24d19c3bcf1ed20c6f77e35b70803e7ba6199d8aadb06ed91822f4327f95cf2932d363c88e7f7fa283b6b210191ee3f3f3a0c1839c5a42039c5592d5970ad5d9d913f8124278efad1390e4b901fe6c2b3dfa869f946e56adec09f4cd'

## 2. OAEP Transform (Encrypt)

In [44]:
# helper functions https://stackoverflow.com/a/30375198/6600660

def int_to_bytes(x: int) -> bytes:
    return x.to_bytes((x.bit_length() + 7) // 8, 'big')

def int_from_bytes(xbytes: bytes) -> int:
    return int.from_bytes(xbytes, 'big')

def xor_bytes(x: bytes, y: bytes) -> bytes:
    """Gibt x ^ y zurück"""
    xi = int_from_bytes(x)
    yi = int_from_bytes(y)
    return int_to_bytes(xi ^ yi)

# Test XOR
assert xor_bytes(bytes([1, 8, 16]), bytes([4, 16, 15])) == bytes([5, 24, 31])

In [77]:
def oaep_encrypt(hash_func, mgf, n: bytes, m: bytes, l: bytes = bytes()):
    # lHash = hash(l), l = leeres Byte Array
    lHash = hash_func(l)

    if len(m) > len(n) - 2 * len(lHash) - 2:
        raise ValueError("m too large!")

    # Zufälliger Seed mit len(seed) = len(lHash)
    seed = bytearray(np.random.bytes(len(lHash)))

    # PS 00..0 sodass len(n) = len(m) + len(PS) + 2 * len(lHash) + 2
    PS = bytearray(len(n) - len(m) - 2 * len(lHash) - 2)

    in_block = lHash + PS + bytes([1]) + m
    mgf_seed = mgf(seed, len(in_block), hash_func)

    # maskedDB = mgf(seed) ^ (lHash + PS + 0x01 + m)
    maskedDB = xor_bytes(mgf_seed, in_block)
    mgf_maskedDB = mgf(maskedDB, len(seed), hash_func)

    # maskedSeed = seed ^ mgf(maskedDB)
    maskedSeed = xor_bytes(seed, mgf_maskedDB)

    # Debug Ausgaben
    print("len(n) =", len(n))
    print("len(m) =", len(m))
    print("len(PS) =", len(PS))
    print("len(lHash) =", len(lHash))

    # Output: 0x00 + maskedSeed + maskedDB
    return bytes([0]) + maskedSeed + maskedDB

# Test mit validem m
n = 808242064728469385653767189449014217949107052233725383595383193397100216381491869385447241469366249460215823319154809183840296738060716935081787424498603981
m = 42
n_bytes = int_to_bytes(n)
m_bytes = int_to_bytes(m)
toenc_bytes = oaep_encrypt(hash_func, mgf1, n_bytes, m_bytes)
toenc_int = int_from_bytes(toenc_bytes)
print(n)
print(toenc_int)
assert toenc_int < n

# Test mit ungültigem m
exc_thrown = False
try:
    oaep_encrypt(hash_func, mgf1, n_bytes, n_bytes)
except ValueError:
    exc_thrown = True
assert exc_thrown

len(n) = 65
len(m) = 1
len(PS) = 22
len(lHash) = 20
808242064728469385653767189449014217949107052233725383595383193397100216381491869385447241469366249460215823319154809183840296738060716935081787424498603981
11659086835643762079576132258464404132802371492794592575087156738771450976499582702587723766236658408112703610531383351598404166641438867087391231422346669


## 3. OAEP Transform (Decrypt)

In [78]:
def oaep_decrypt(hash_func, mgf, n: bytes, m: bytes) -> bytes:
    """Kehrt OAEP-Transformation um.

    Args:
        hash_func (function bytes -> bytes): Hash-Funktion
        mgf (function): Mask Generating Function
        n (bytes): RSA-Modul
        m (bytes): Ausgabe der OAEP-Transformation.

    Returns:
        Eingabe der OAEP-Transformation.
    """
    # Länge der Ausgabe der Hashfunktion
    h = len(hash_func(bytes()))

    # m = 0x00 (Länge 1) + maskedSeed (Länge h) + maskedDB
    maskedSeed = m[1:(1+h)]
    maskedDB = m[(1+h):]

    mgf_maskedDB = mgf(maskedDB, h, hash_func)

    #     maskedSeed = seed ^ mgf(maskedDB)
    # <-> seed = maskedSeed ^ mgf(maskedDB)
    seed = xor_bytes(maskedSeed, mgf_maskedDB)
    mgf_seed = mgf(seed, len(maskedDB), hash_func)

    #     maskedDB = mgf(seed) ^ (lHash + PS + 0x01 + m)
    # <-> (lHash + PS + 0x01 + m) = maskedDB ^ mgf(seed)
    in_block = xor_bytes(maskedDB, mgf_seed)

    # PS(00..0) + 0x01 + m
    zeros_one_m = in_block[h:]
    # Extrahiere m (beginnt hinter dem ersten Byte mit Wert 1)
    for i in range(len(zeros_one_m)):
        if zeros_one_m[i] == 1:
            return zeros_one_m[(i+1):]
    
    # Wenn hier angelangt: Ungültiges Format (kein 1-Byte)
    raise ValueError("Ungültiges Format")


## 4. Tests

In [83]:
# Gesamttest mit Text
m_bytes = b"To be or not to be"
enc_bytes = oaep_encrypt(hash_func, mgf1, n_bytes, m_bytes)
dec_bytes = oaep_decrypt(hash_func, mgf1, n_bytes, enc_bytes)
print(m_bytes)
print(dec_bytes)
assert m_bytes == dec_bytes

len(n) = 65
len(m) = 18
len(PS) = 5
len(lHash) = 20
b'To be or not to be'
b'To be or not to be'


In [82]:
# Gesamttest mit Zahl
m_bytes = int_to_bytes(1234567890)
enc_bytes = oaep_encrypt(hash_func, mgf1, n_bytes, m_bytes)
dec_bytes = oaep_decrypt(hash_func, mgf1, n_bytes, enc_bytes)
print(m_bytes)
print(dec_bytes)
assert m_bytes == dec_bytes

len(n) = 65
len(m) = 4
len(PS) = 19
len(lHash) = 20
b'I\x96\x02\xd2'
b'I\x96\x02\xd2'


In [84]:
# Gesamttest mit leerer Nachricht
m_bytes = bytes()
enc_bytes = oaep_encrypt(hash_func, mgf1, n_bytes, m_bytes)
dec_bytes = oaep_decrypt(hash_func, mgf1, n_bytes, enc_bytes)
print(m_bytes)
print(dec_bytes)
assert m_bytes == dec_bytes

len(n) = 65
len(m) = 0
len(PS) = 23
len(lHash) = 20
b''
b''


In [1]:
bytes() + bytes()

b''